##데이터 전처리

###bond_report

In [ ]:
pip install ekonlpy

In [2]:
#라이브러리
import re
import time
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [15]:
import pandas as pd
bond_df=pd.read_csv("./Bond_report_naver.csv")
bond_report=pd.DataFrame(set([(bond_df.loc[i,"0"], bond_df.loc[i,"1"]) for i in range(len(bond_df))]))    # 중복 행 제거
bond_report.columns=["DATE", "CONTENTS"]

In [16]:
bond_report["CONTENTS"]=[eval(bond_report.loc[i,"CONTENTS"]) for i in range(len(bond_report))] #str → list
report_content=[]
for doc in bond_report["CONTENTS"]:
    idx=2       #3번째 줄부터 저장
    content=[]
    while idx<len(doc)-1:
        if  doc[idx].startswith("자료 출처:") or doc[idx].startswith("자료:") or doc[idx].startswith("그림") or doc[idx].startswith("도표") or len(doc[idx])<2: #표, 그래프, 자료출처 문장 제외
            idx+=1
        elif doc[idx].startswith("(단위:"): #(단위: ) = 표 내용 → 다음 2문장까지 같이 삭제
            idx+=3
        elif len(re.findall(r"\d+(\.\d+)?", doc[idx]))> len(re.findall("[가-힣a-zA-Z]+", doc[idx])): #숫자가 문자보다 많을경우 문장 삭제
            idx+=1
        elif doc[idx].startswith("[ Compliance Notice ]"):  #뒷내용 삭제
          break
        else:
            content.append(doc[idx])
            idx+=1
    report_content.append(content)

bond_report["CLEANED_CONTENTS"]=report_content

In [17]:
#텍스트 CLEANED_CONTENTS 한 뭉치로 변환
bond_report["CLEANED_CONTENTS"]=["".join(bond_report.loc[i,"CLEANED_CONTENTS"]) for i in range(len(bond_report))]

#텍스트 정규표현식으로 정제
def clean(df, regex, replace):
    df["CLEANED_CONTENTS"]=df["CLEANED_CONTENTS"].replace(regex,replace, regex=True)

clean(bond_report, r"[^.]*니다 ?[.]", "")      #'니다'로 끝나는 문장 삭제
clean(bond_report, r'\[[^]]*\]', "")           #[] 삭제
clean(bond_report, r'\S+@\S+', "")             #이메일
clean(bond_report, r'.\d+-\d+.*?',"")          #전화번호
clean(bond_report, r'\d+\.\d*',"`")
bond_report

,DATE,CONTENTS,CLEANED_CONTENTS
0,2010-10-05,"[2010년 10월 05일, FFiixxeedd IInnccoommee Daily ...",<summary>Fixed Income Strategist 오창섭 (cid:1) 금...
1,2021-12-01,"[2021년 12월 1일 I Global Asset Research, CrediVi...",오비이락(烏飛梨落)Issue Comment금통위와 오미크론의 조우(遭遇) 전주 금...
2,2019-07-24,"[2019년 07월 24읷 I Global Asset Research, CrediV...",제한적 강세 젂망Issue Comment신용채권의 강세 기조는 이어지지만 추가 스프...
3,2023-08-28,"[[자산배분/해외크레딧 Weekly], 잇단 신용 강등에도 은행채 스프레드 하락, ...",자산배분/해외크레딧 Weekly I `.28해외크레딧/자산배분 이영주 RA 하형민...
4,2017-10-11,"[엎친데 덮친 격; 금리가 오르고 있다., Issue Comment, Credit,...","CreditCredit 김상만, Analyst 김상만-7508크레딧스프레드 동향..."
...,...,...,...
5602,2010-05-12,"[www.ibks.com, 5 월 금통위 Watch 채권전략, 2010년 5월 12...",2010년 5월 12일Fixed Income Strategist 오창섭 ☎ 02 6...
5603,2019-05-15,"[2019년 5월 14읷 I Global Asset Research, Catch F...",안전자산으로 도망 또 도망Investment Strategy북미 주식펀드 대규모 유...
5604,2018-05-04,"[Fixed Income BOND+, 2018. 05. 04, 5월 금통위 금리인상...",5월 금통위 금리인상 소수의견 예상4월 금통위 의사록(5/2일 공개)에 대한 평가F...
5605,2010-04-22,"[www.ibks.com, 채권분석, Fixed Income Issue, 2010년...",Fixed Income Issue2010년 4월 22일미국 금융규제에 따른 채권시장...


In [20]:
analysts=["안재균", "정혜진", "김상만", "김상인", "김지나", "이정훈", "박민영", "김상훈", "김현지", "권재형", "구혜영", "김민형", "이경록", "박승진", "김정형", "안예하", "박진영", "안주영", "전병하", "오창섭", "윤일광", "서철수", "윤여삼","유욱재","모승규", "김성수"]
for name in analysts:
  bond_report["CLEANED_CONTENTS"]=bond_report["CLEANED_CONTENTS"].replace(name,"", regex=True)

In [21]:
#내용확인
bond_report.loc[3541,"CLEANED_CONTENTS"]

'한화 FOMC Review자신이 없으면 목소리가 커지는 법▶ 채권전략   / 일단은 동결6월 FOMC, 기준금리 `%로 만장일치 동결주요 채권금리 추이(%) Powell 의장은 추가적인 데이터와 정보를 확인할 시간이 필요함을 기준금리미국기준금리동결 배경으로 설명. 또한, 금번 결정이 인상 종료가 아님을 강조하면서 거의` 미국2년` 미국10년 모든 위원들이 추가 기준금리 인상이 적절하다는데 동의했음을 언급내용은 강경했지만…` 기자회견은 예상했던대로 다분히 매파적이었음. 다만, 추가 인상을 절대 다` 수가 동의하지만 다음 결정은 다음 회의에서 할 것이고(기자회견), 금번 결정은 추가적인 데이터와 정보를 확인하기 위함(성명서) 등 인상 중단에 대한가능성을 열어놓았다는 판단성명서 내용은 5월 대비 큰 변화 부재. ‘기준금리 인상에 있어 통화정책 파급효과를 고려할 것’ 문구도 유지. 기준금리 점도표 중간값은 `%에서`%로 상향 조정. 18명 위원 중 12명이 연내 추가 2회 인상을 예상연내 동결 전망 유지상향 조정된 근원 물가, 경제 성장률 전망치를 고려하면 7월 또는 그 이후인상 가능성이 높아진 것은 사실. 그러나 정책의 파급효과를 지켜보아야 한다는 기조는 유지 중이고, 추가 인상을 기정사실처럼 강조했지만 막상 확답에 가까운 언급은 없었음향후 2년 동안 기준금리 인하가 없을 것이라는 발언에도 주목할 필요. 이제부터 ‘Higher for Longer’ 성공을 위해서는 기준금리 인상보다 유지가 더 중요해진 상황데이터에 기반한 정책 결정 꾸준히 강조 중인데, 당사는 물가, 고용의 둔화추세는 바뀌지 않을 것으로 전망. 연내 기준금리 동결 전망 유지'

In [22]:
from ekonlpy import Mecab
with open("./stopword.txt", "r") as f:
  stopword=f.read().splitlines()

mecab=Mecab()
pos_list=[mecab.replace_synonyms(bond_report.loc[row, "CLEANED_CONTENTS"]) for row in range(len(bond_report))]    #tokenize
pos_tag=[[token for token in row if token[1] in ['NNG', "VA", "VAX", "MAG", "V"] and token[0] not in stopword] for row in pos_list]   #stopword 제외, 특정 품사만 저장
bond_report["TOKEN"]=pos_tag

In [23]:
bond_report.head()

,DATE,CONTENTS,CLEANED_CONTENTS,TOKEN
0,2010-10-05,"[2010년 10월 05일, FFiixxeedd IInnccoommee Daily ...",<summary>Fixed Income Strategist (cid:1) 금일 시...,"[(금일, NNG), (시장, NNG), (금리, NNG), (저점, NNG), (..."
1,2021-12-01,"[2021년 12월 1일 I Global Asset Research, CrediVi...",오비이락(烏飛梨落)Issue Comment금통위와 오미크론의 조우(遭遇) 전주 금...,"[(오비이락, NNG), (烏飛梨落, NNG), (금통위, NNG), (오미크론, ..."
2,2019-07-24,"[2019년 07월 24읷 I Global Asset Research, CrediV...",제한적 강세 젂망Issue Comment신용채권의 강세 기조는 이어지지만 추가 스프...,"[(제한적, VAX), (강세, NNG), (망, NNG), (크레딧, NNG), ..."
3,2023-08-28,"[[자산배분/해외크레딧 Weekly], 잇단 신용 강등에도 은행채 스프레드 하락, ...",자산배분/해외크레딧 Weekly I `.28해외크레딧/자산배분 이영주 RA 하형민...,"[(자산배분, NNG), (해외, NNG), (크레딧, NNG), (해외, NNG)..."
4,2017-10-11,"[엎친데 덮친 격; 금리가 오르고 있다., Issue Comment, Credit,...","CreditCredit , Analyst -7508크레딧스프레드 동향 및 전망연...","[(신용스프레드, NNG), (전망, NNG), (연휴, NNG), (전, NNG)..."


###kyobo_report

In [4]:
kyobo_df=pd.read_csv("./Bond_report_kyobo.csv")
kyobo_report=pd.DataFrame(set([(kyobo_df.loc[i,"0"], kyobo_df.loc[i,"1"]) for i in range(len(kyobo_df))]))    #중복 행 제거
kyobo_report.columns=["DATE", "CONTENTS"]

In [5]:
kyobo_report["CONTENTS"]=[eval(kyobo_report.loc[i,"CONTENTS"]) for i in range(len(kyobo_report))]
report_content=[]
for doc in kyobo_report["CONTENTS"]:
    idx=0
    content=[]
    while idx<len(doc)-1:
        if idx==0:
            if doc[0].startswith("Daily") or doc[0].startswith("Fixed") or doc[0].startswith("발간일자"):
                idx+=3
            elif doc[0].startswith("채권전략 연구위원"):
                idx+=6
            else:
                content.append(doc[0])
                idx+=1

        if doc[idx].startswith("국내주요금리 (%, bp)"):  #표 내용 제외
          break
        elif doc[idx].startswith("자료:") or doc[idx].startswith("[도표") or doc[idx].startswith("주:") or doc[idx].startswith("도표") or len(doc[idx])<2: #표, 그래프, 자료출처 문장 제외
            idx+=1
        elif len(re.findall(r"\d+(\.\d+)?", doc[idx]))> len(re.findall("[가-힣a-zA-Z]+", doc[idx])):
            idx+=1
        else:
            content.append(doc[idx])
            idx+=1
    report_content.append(content)

kyobo_report["CLEANED_CONTENTS"]=report_content

In [10]:
#텍스트 내용 한 뭉치로 변환
kyobo_report["CLEANED_CONTENTS"]=["".join(kyobo_report.loc[i,"CLEANED_CONTENTS"]) for i in range(len(kyobo_report))]

#텍스트 정규표현식으로 정제
def clean(df, regex, replace):
    df["CLEANED_CONTENTS"]=df["CLEANED_CONTENTS"].replace(regex,replace, regex=True)

clean(kyobo_report, r"[^.]*니다 ?[.]", "")      #'니다'로 끝나는 문장 삭제
clean(kyobo_report, r'\[[^]]*\]', "")           #[] 삭제
clean(kyobo_report, r'\S+@\S+', "")             #이메일
clean(kyobo_report, r'.\d+-\d+.*?',"")          #전화번호
clean(kyobo_report, r'.\d+_\d+.*?',"")          #전화번호
clean(kyobo_report, r'\d+\.\d*',"`")            #소수 → `
clean(kyobo_report, r'■', ".")
clean(kyobo_report, r'▪', ".")

#이름 삭제
analysts=['백윤민','권한욱','이슬비','정윤정','공동락', '이정준']
for name in analysts:
  kyobo_report["CLEANED_CONTENTS"]=kyobo_report["CLEANED_CONTENTS"].replace(name,"", regex=True)

In [11]:
#내용확인
kyobo_report.head()

,DATE,CONTENTS,CLEANED_CONTENTS
0,2015-06-11,"[채권전략 연구위원 권한욱, 02_3771_9633 / hwkwon@iprovest...","상하6월 금통위 기준금리 인하 또는 소수의견 증대 예상 6월 금통위, 수출 부진, ..."
1,2014-03-20,"[채권전략 연구위원 권한욱, 02_3771_9633 / hwkwon@iprovest...",큰 폭 상승한 미국채 금리 및 달러 인덱스. 전일 동향- 이주열 한은 총재 내정자 ...
2,2013-02-08,"[채권전략 연구위원 권한욱, 02_3771_9633 / hwkwon@iprovest...",. 전일 동향- 외국인 국채선물 순매수가 1 만 계약을 상회하는 등 매수 심리 지지...
3,2005-05-09,"[2005. 5. 9 (월), 교보 주간 Bond Ma p, Best Investm...",` ` 9 (월)교보 주간 Bond Ma pBest Investment Partne...
4,2023-02-14,"[Fixed Income 백윤민, Daily 02_3771_9175 / yoonmi...",Fixed Income_9733 / Comment Today's Key Chart...


In [13]:
from ekonlpy import Mecab
with open("./stopword.txt", "r") as f:
  stopword=f.read().splitlines()

mecab=Mecab()
pos_list=[mecab.replace_synonyms(kyobo_report.loc[row, "CLEANED_CONTENTS"]) for row in range(len(kyobo_report))]
pos_tag=[[token for token in row if token[1] in ['NNG', "VA", "VAX", "MAG", "V"] and token[0] not in stopword] for row in pos_list]
kyobo_report["TOKEN"]=pos_tag

In [14]:
kyobo_report.head()

,DATE,CONTENTS,CLEANED_CONTENTS,TOKEN
0,2015-06-11,"[채권전략 연구위원 권한욱, 02_3771_9633 / hwkwon@iprovest...","상하6월 금통위 기준금리 인하 또는 소수의견 증대 예상 6월 금통위, 수출 부진, ...","[(금통위, NNG), (금리, NNG), (인하, NNG), (소수의견, NNG)..."
1,2014-03-20,"[채권전략 연구위원 권한욱, 02_3771_9633 / hwkwon@iprovest...",큰 폭 상승한 미국채 금리 및 달러 인덱스. 전일 동향- 이주열 한은 총재 내정자 ...,"[(큰, VA), (폭, NNG), (상승, NNG), (미국, NNG), (금리,..."
2,2013-02-08,"[채권전략 연구위원 권한욱, 02_3771_9633 / hwkwon@iprovest...",. 전일 동향- 외국인 국채선물 순매수가 1 만 계약을 상회하는 등 매수 심리 지지...,"[(외국인, NNG), (국채선물, NNG), (순매수, NNG), (계약, NNG..."
3,2005-05-09,"[2005. 5. 9 (월), 교보 주간 Bond Ma p, Best Investm...",` ` 9 (월)교보 주간 Bond Ma pBest Investment Partne...,"[(교보, NNG), (암중모색, NNG), (모멘텀, NNG), (시중, NNG)..."
4,2023-02-14,"[Fixed Income 백윤민, Daily 02_3771_9175 / yoonmi...",Fixed Income_9733 / Comment Today's Key Chart...,"[(미, NNG), (미국, NNG), (채권시장, NNG), (미국, NNG), ..."
